In [808]:
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 10, 8
import pandas as pd
from sklearn.metrics import accuracy_score

In [809]:
df = pd.read_csv('C:\PY\winequality-red.csv')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [810]:
#We prepare our data for training, namely:
#1) We load the general necessary libraries
#2) Select the target variable "quality" from
#3) Through.drop()) we separate the independent variables from the target
#4) We divide the dataset into a test sample and a training sample (take 20%)

In [811]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [812]:
y = df['quality']

In [813]:
X = df.drop('quality',axis=1)

In [814]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=17)

# KNN

In [823]:
from sklearn.neighbors import KNeighborsClassifier

In [824]:
knn = KNeighborsClassifier(n_jobs=-1)

In [825]:
X.shape , y.shape

((1599, 11), (1599,))

In [826]:
knn_params ={'n_neighbors': np.arange(1,24)}

In [827]:
knn_search = GridSearchCV(knn, knn_params, cv=5, return_train_score=True,n_jobs=-1)

In [828]:
%time
knn_search.fit(X_train, y_train)

Wall time: 0 ns


GridSearchCV(cv=5, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=-1,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=-1,
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring=None, verbose=0)

In [829]:
knn_search.best_params_ , knn_search.best_score_

({'n_neighbors': 1}, 0.569969362745098)

In [830]:
knn_score= knn_search.best_score_
knn_predict = knn_search.predict(X_test)

In [831]:
knn_acc = accuracy_score(y_test,knn_predict)
knn_acc

0.50625

# Linear Regression

In [832]:
from sklearn.linear_model import LinearRegression

In [833]:
lr = LinearRegression()

In [834]:
%%time
lr.fit(X_train,y_train)

Wall time: 2.97 ms


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [835]:
lr_predict = lr.predict(X_test)

In [836]:
lr_predict = lr.predict(X_test)
lr_acc = accuracy_score(y_test,np.around(lr_predict)) 
lr_acc

0.571875

# DecisionTree

In [837]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score
import graphviz
from sklearn.tree import export_graphviz

In [838]:
first_tree = DecisionTreeClassifier(random_state=17)

In [839]:
tree_params = {'max_depth': np.arange(1, 20), 'min_samples_leaf':np.arange(1,20)}

In [840]:
tree_grid = GridSearchCV(first_tree,tree_params,cv=5)
cross_val_score(first_tree,X_train,y_train,cv=5)

array([0.58203125, 0.60546875, 0.5546875 , 0.6015625 , 0.62352941])

In [841]:
%%time
tree_grid.fit(X_train, y_train)

Wall time: 14 s


GridSearchCV(cv=5, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=17,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9,

In [842]:
tree_predict = tree_grid.predict(X_test)

In [843]:
tree_acc = accuracy_score(y_test, tree_predict)
tree_acc

0.55625

In [844]:
#dot = export_graphviz(tree_grid.best_estimator_, out_file=None ,field=None)

In [845]:
#graph = graphviz.Source(dot)
#graph

## Logistic Regression

In [846]:
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

In [847]:
logr= LogisticRegression(penalty='l2',max_iter=900, random_state=17,n_jobs=-1)
logr_params = {'C': np.linspace(0,1,11)}

In [848]:
logr_search = GridSearchCV(logr,logr_params,n_jobs=-1,cv=3)

In [849]:
%%time
logr_search.fit(X,y)

Wall time: 8.15 s


GridSearchCV(cv=3, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=900, multi_class='auto',
                                          n_jobs=-1, penalty='l2',
                                          random_state=42, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [850]:
logr_search.best_params_


{'C': 0.2}

In [851]:
logr_score=logr_search.best_score_
logr_predict = logr_search.predict(X_test)

In [852]:
logr_acc = accuracy_score(y_test,logr_predict)
logr_acc

0.559375

# Ranodm Forest

In [853]:
from sklearn.ensemble import RandomForestClassifier

In [854]:
rf_reg = RandomForestClassifier(random_state=17,n_jobs=-1)

In [855]:
params = dict(n_estimators= [100]  ,max_depth=np.arange(6,13), max_features=np.arange(1,6),\
           bootstrap =['True','False'],min_samples_leaf=np.arange(1,3))
par


{'n_estimators': (100, 100),
 'max_depth': array([ 6,  7,  8,  9, 10, 11, 12]),
 'max_features': array([1, 2, 3, 4, 5]),
 'bootstrap': ['True', 'False'],
 'min_samples_leaf': array([1, 2])}

In [856]:
rf_greed = GridSearchCV(estimator=rf_reg,param_grid=params,n_jobs=-1,cv=3,return_train_score=True)

In [857]:
%%time
rf_greed.fit(X_train,y_train)

Wall time: 38.6 s


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=-1,
                                              oob_score=False, random_state=42,
                                    

In [858]:
rf_greed.best_params_

{'bootstrap': 'True',
 'max_depth': 10,
 'max_features': 3,
 'min_samples_leaf': 1,
 'n_estimators': 100}

In [859]:
rf_score = rf_greed.best_score_
rf_predict = rf_greed.predict(X_test)

In [860]:
rf_acc = accuracy_score(y_test,rf_predict)
rf_acc

0.64375

# GradientBoostingCLassifier

In [861]:
from sklearn.ensemble import GradientBoostingClassifier

In [862]:
gb = GradientBoostingClassifier(max_features=11,min_samples_leaf=1,max_depth=6,random_state=42)

In [863]:
gb_params = {'n_estimators': np.arange(30,50)}

In [864]:
gb_search = GridSearchCV(gb,gb_params,cv=5,n_jobs=-1)

In [865]:
%%time
gb_search.fit(X_train, y_train)

Wall time: 34.7 s


GridSearchCV(cv=5, error_score=nan,
             estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=6,
                                                  max_features=11,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_change=None,
           

In [866]:
gb_score = gb_search.best_score_
gb_score
gb_predict = gb_search.predict(X_test)

In [867]:
gb_acc=accuracy_score(y_test,gb_predict)
gb_acc

0.65625

# Comparison of models by their accuracy

In [873]:
data = pd.DataFrame([['KNN',knn_acc],
                     ['DesicionTree',tree_acc],
                     ['LinearRegression', lr_acc],
                     ['LogisticRegression', logr_acc],
                     ['GradientBoosting',gb_acc],
                     ['Random Forest', rf_acc]],
                    columns = ['Model name','Accuracy'])

                
data

,Model name,Accuracy
0,KNN,0.506250
1,DesicionTree,0.556250
2,LinearRegression,0.571875
3,LogisticRegression,0.559375
4,GradientBoosting,0.656250
5,Random Forest,0.643750


In [874]:
data = data.sort_values(by=['Accuracy'], ascending=False)  
data['Accuracy']*=100
data

,Model name,Accuracy
4,GradientBoosting,65.6250
5,Random Forest,64.3750
2,LinearRegression,57.1875
3,LogisticRegression,55.9375
1,DesicionTree,55.6250
0,KNN,50.6250


In [871]:
#findings

In [872]:
#The Gradient Boosting model turned out to be the best. 
#It has an accuracy on the test sample of 65.6%.
#The fastest model was KNN, but it also has the lowest accuracy rate (50.6%)